In [1]:
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def fetch_and_extract(url):
    """Fetch a PDF from a URL and extract text."""
    response = requests.get(url)
    with open("temp.pdf", "wb") as f:
        f.write(response.content)
    reader = PdfReader("temp.pdf")
    text = "".join(page.extract_text() or "" for page in reader.pages)
    return text

# Example URL (replace with a real one)
url = "https://universitycounsel.ubc.ca/files/2022/05/Health-and-Safety-Policy_SC1.pdf"
document_text = fetch_and_extract(url)

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(document_text)
print(f"Split into {len(chunks)} chunks")

Split into 118 chunks


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store
vector_store = FAISS.from_texts(chunks, embeddings)
print("Vector store created")

/var/folders/gq/2v9xsw650h5cpd24shf5w32w0000gn/T/ipykernel_96777/4080657736.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created


In [8]:
import requests
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class OpenRouterLLM(LLM):
    api_key: str  # Define as a class-level Pydantic field
    model: str = "meta-llama/llama-3.1-8b-instruct"  # Default model

    def __init__(self, api_key: str, model: str = None, **kwargs):
        # Pass api_key to the parent class (LLM) via super().__init__()
        super().__init__(api_key=api_key, **kwargs)
        if model:
            self.model = model  # Override default model if provided

    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 500
        }
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", 
                                json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model}

# Initialize with your API key
api_key = "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"
llm = OpenRouterLLM(api_key=api_key)

In [9]:
# Test code
llm = OpenRouterLLM(api_key="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")
response = llm("Hello, how are you?")
print(response)

/var/folders/gq/2v9xsw650h5cpd24shf5w32w0000gn/T/ipykernel_96777/2260345510.py:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm("Hello, how are you?")


I'm just a computer program, so I don't have feelings, but thanks for asking! How can I assist you today?


In [10]:
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import streamlit as st

# Fixed OpenRouterLLM class
class OpenRouterLLM(LLM):
    api_key: str
    model: str = "meta-llama/llama-3.1-8b-instruct"

    def __init__(self, api_key: str, model: str = None, **kwargs):
        super().__init__(api_key=api_key, **kwargs)
        if model:
            self.model = model

    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 500
        }
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", 
                                json=payload, headers=headers)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model}

# Fetch and extract
def fetch_and_extract(url):
    response = requests.get(url)
    with open("temp.pdf", "wb") as f:
        f.write(response.content)
    reader = PdfReader("temp.pdf")
    return "".join(page.extract_text() or "" for page in reader.pages)

# Main app
st.title("Policy Analyzer")
url = st.text_input("Enter Policy URL", value="https://strategicplan.ubc.ca/wp-content/uploads/2019/09/2018_UBC_Strategic_Plan_Full-20180425.pdf")
query = st.text_input("Ask a Question", value="What are the strategic goals?")
if st.button("Analyze"):
    with st.spinner("Fetching and analyzing..."):
        # Prepare document
        document_text = fetch_and_extract(url)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = text_splitter.split_text(document_text)

        # Embed
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        vector_store = FAISS.from_texts(chunks, embeddings)

        # RAG with OpenRouter
        llm = OpenRouterLLM(api_key="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")
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vector_store.as_retriever(search_kwargs={"k": 3})
        )
        result = qa_chain.run(query)
        st.write("Answer:", result)

2025-03-25 12:27:07.832 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-25 12:27:07.833 Session state does not function when running a script without `streamlit run`


In [15]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Set up retry strategy
retry_strategy = Retry(
    total=10,
    backoff_factor=2,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)

url = "https://www2.gov.bc.ca/gov/content/education-training/post-secondary-education/institution-resources-administration/accountability-framework"
response = http.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

pdf_links = []
for link in soup.find_all('a', href=True):
    href = link.get('href')
    if '.pdf' in href.lower():
        # Convert relative URLs to absolute ones
        full_url = urljoin(url, href)
        pdf_links.append(full_url)

print(pdf_links)

ConnectionError: HTTPSConnectionPool(host='www2.gov.bc.ca', port=443): Max retries exceeded with url: /gov/content/education-training/post-secondary-education/institution-resources-administration/accountability-framework (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))

In [ ]:
<div class="panel-text">
<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/bcit_iapr.pdf" target="_blank">British Columbia Institute of Technology (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/cam_iapr.pdf" target="_blank">Camosun College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/capu_iapr.pdf" target="_blank">Capilano University (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/cmtn_iapr.pdf" target="_blank">Coast Mountain College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/cnc_iapr.pdf" target="_blank">College of New Caledonia (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/cotr_iapr.pdf" target="_blank">College of the Rockies (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/doug_iapr.pdf" target="_blank">Douglas College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/ecu_iapr.pdf" target="_blank">Emily Carr University of Art and Design (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/jibc_iapr.pdf" target="_blank">Justice Institute of British Columbia (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/kpu_iapr.pdf" target="_blank">Kwantlen Polytechnic University (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/lang_iapr.pdf" target="_blank">Langara College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/nvit_iapr.pdf" target="_blank">Nicola Valley Institute of Technology (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/nic_iapr.pdf" target="_blank">North Island College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/nlc_iapr.pdf" target="_blank">Northern Lights College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/okan_iapr.pdf" target="_blank">Okanagan College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/rru_iapr.pdf" target="_blank">Royal Roads University (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/sel_iapr.pdf" target="_blank">Selkirk College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/sfu_iapr.pdf" target="_blank">Simon Fraser University (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/tru_iapr.pdf" target="_blank">Thompson Rivers University (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/ubc_iapr.pdf" target="_blank">University of British Columbia (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/unbc_iapr.pdf" target="_blank">University of Northern British Columbia (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/ufv_iapr.pdf" target="_blank">University of the Fraser Valley (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/uvic_iapr.pdf" target="_blank">University of Victoria (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/vcc_iapr.pdf" target="_blank">Vancouver Community College (PDF)</a></p>

<p><a href="https://www2.gov.bc.ca/assets/gov/education/post-secondary-education/institution-resources-administration/accountability-framework/iapr/viu_iapr.pdf" target="_blank">Vancouver Island University (PDF)</a></p>
</div>